In [ ]:
!pip install batch-face opencv-python

!mkdir -p checkpoints

!wget -O checkpoints/mobilenet.pth "https://huggingface.co/spaces/kaushikpandav/Lipsync_Antriksh_AI/resolve/a24a1bf9e93a245a80fcf33a7178576958e53e39/checkpoints/mobilenet.pth"

import cv2
import math
import numpy as np
import csv
from batch_face import RetinaFace
from google.colab import files


In [ ]:
uploaded = files.upload()
video_filename = list(uploaded.keys())[0]

detector = RetinaFace(gpu_id=0, model_path="checkpoints/mobilenet.pth",
                      network="mobilenet")

cap = cv2.VideoCapture(video_filename)
if not cap.isOpened():
    raise IOError("Error opening video file")

fps = cap.get(cv2.CAP_PROP_FPS)

max_faces = 0

csv_filename = "face_group_multiple.csv"
csv_file = open(csv_filename, mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["frame", "face_id", "x1", "y1", "x2", "y2"])

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    detections = detector([frame], threshold=0.94)

    if detections and detections[0] is not None:
        face_detections = detections[0]
        face_count = len(face_detections)
        if face_count > max_faces:
            max_faces = face_count

        for face_id, detection in enumerate(face_detections):
            box, landmarks, score = detection
            box = list(map(int, box))
            x1, y1, x2, y2 = box
            csv_writer.writerow([frame_count, face_id, x1, y1, x2, y2])

cap.release()
csv_file.close()

from google.colab import files
files.download(csv_filename)

print("Processed {} frames.".format(frame_count))
print(f"No of faces (max in a frame): {max_faces}")
print("CSV file with face bounding boxes saved as:", csv_filename)